# 🤖 Modelo de Machine Learning - Predição de Preços

**Autor:** Marcos Paulo Roriz Lima Reis  
**RA:** 22007534  
**Email:** marcos.paulor@sempreceub.com  
**Curso:** Engenharia da Computação - UniCEUB  

## 🎯 Objetivos
- Implementar modelos de regressão para predição de preços
- Comparar performance de diferentes algoritmos
- Avaliar qualidade das predições
- Fazer predições para novos imóveis

In [ ]:
# Importação das bibliotecas
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Scikit-learn
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

# Configuração dos gráficos
plt.style.use('default')
sns.set_palette('husl')
plt.rcParams['figure.figsize'] = (12, 8)

print("✅ Bibliotecas importadas com sucesso!")

In [ ]:
# Carregamento dos dados tratados
data_path = Path('../data/imoveis_rurais_tratados.csv')
df = pd.read_csv(data_path)

print(f"📊 Dataset carregado com {len(df)} registros")
print(f"📏 Dimensões: {df.shape}")
df.head()

In [ ]:
# Preparação dos dados para modelagem
print("🔧 Preparando dados para modelagem...")

# Selecionar features numéricas
numeric_features = df.select_dtypes(include=[np.number]).columns.tolist()
if 'preco' in numeric_features:
    numeric_features.remove('preco')  # Remover target

print(f"📋 Features numéricas disponíveis: {numeric_features}")

# Verificar dados ausentes
missing_data = df[numeric_features + ['preco']].isnull().sum()
print(f"\n🔍 Dados ausentes:")
print(missing_data[missing_data > 0])

# Remover registros com dados ausentes
df_model = df[numeric_features + ['preco']].dropna()
print(f"\n📊 Dataset para modelagem: {len(df_model)} registros")

In [ ]:
# Definir variáveis X e y
X = df_model[numeric_features]
y = df_model['preco']

print(f"📋 Features (X): {list(X.columns)}")
print(f"🎯 Target (y): preco")
print(f"📊 Shape X: {X.shape}")
print(f"📊 Shape y: {y.shape}")

# Estatísticas das features
print("\n📈 Estatísticas das Features:")
print(X.describe())

In [ ]:
# Divisão em treino e teste
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

print(f"📊 Conjunto de treino: {X_train.shape[0]} registros")
print(f"📊 Conjunto de teste: {X_test.shape[0]} registros")
print(f"📊 Proporção treino/teste: {X_train.shape[0]/X_test.shape[0]:.1f}:1")

In [ ]:
# Normalização dos dados
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print("⚖️ Dados normalizados com StandardScaler")
print(f"📊 Média das features (treino): {np.mean(X_train_scaled, axis=0)}")
print(f"📊 Desvio padrão das features (treino): {np.std(X_train_scaled, axis=0)}")

In [ ]:
# Modelo 1: Regressão Linear
print("🤖 Treinando Modelo de Regressão Linear...")

lr_model = LinearRegression()
lr_model.fit(X_train_scaled, y_train)

# Predições
y_pred_lr_train = lr_model.predict(X_train_scaled)
y_pred_lr_test = lr_model.predict(X_test_scaled)

# Métricas
lr_r2_train = r2_score(y_train, y_pred_lr_train)
lr_r2_test = r2_score(y_test, y_pred_lr_test)
lr_rmse_train = np.sqrt(mean_squared_error(y_train, y_pred_lr_train))
lr_rmse_test = np.sqrt(mean_squared_error(y_test, y_pred_lr_test))
lr_mae_test = mean_absolute_error(y_test, y_pred_lr_test)

print(f"\n📈 Regressão Linear - Resultados:")
print(f"   R² Treino: {lr_r2_train:.4f}")
print(f"   R² Teste: {lr_r2_test:.4f}")
print(f"   RMSE Treino: R$ {lr_rmse_train:,.2f}")
print(f"   RMSE Teste: R$ {lr_rmse_test:,.2f}")
print(f"   MAE Teste: R$ {lr_mae_test:,.2f}")

In [ ]:
# Modelo 2: Random Forest
print("🌲 Treinando Modelo Random Forest...")

rf_model = RandomForestRegressor(
    n_estimators=100, 
    random_state=42,
    max_depth=10,
    min_samples_split=5
)
rf_model.fit(X_train, y_train)  # Random Forest não precisa de normalização

# Predições
y_pred_rf_train = rf_model.predict(X_train)
y_pred_rf_test = rf_model.predict(X_test)

# Métricas
rf_r2_train = r2_score(y_train, y_pred_rf_train)
rf_r2_test = r2_score(y_test, y_pred_rf_test)
rf_rmse_train = np.sqrt(mean_squared_error(y_train, y_pred_rf_train))
rf_rmse_test = np.sqrt(mean_squared_error(y_test, y_pred_rf_test))
rf_mae_test = mean_absolute_error(y_test, y_pred_rf_test)

print(f"\n🌲 Random Forest - Resultados:")
print(f"   R² Treino: {rf_r2_train:.4f}")
print(f"   R² Teste: {rf_r2_test:.4f}")
print(f"   RMSE Treino: R$ {rf_rmse_train:,.2f}")
print(f"   RMSE Teste: R$ {rf_rmse_test:,.2f}")
print(f"   MAE Teste: R$ {rf_mae_test:,.2f}")

In [ ]:
# Comparação dos modelos
comparison_data = {
    'Modelo': ['Regressão Linear', 'Random Forest'],
    'R² Treino': [lr_r2_train, rf_r2_train],
    'R² Teste': [lr_r2_test, rf_r2_test],
    'RMSE Teste': [lr_rmse_test, rf_rmse_test],
    'MAE Teste': [lr_mae_test, rf_mae_test]
}

comparison_df = pd.DataFrame(comparison_data)
print("🏆 Comparação dos Modelos:")
print("=" * 60)
print(comparison_df.to_string(index=False, float_format='%.4f'))

# Identificar melhor modelo
best_model_idx = comparison_df['R² Teste'].idxmax()
best_model_name = comparison_df.loc[best_model_idx, 'Modelo']
print(f"\n🥇 Melhor modelo: {best_model_name}")

In [ ]:
# Visualização das predições
fig, axes = plt.subplots(1, 2, figsize=(15, 6))

# Regressão Linear
axes[0].scatter(y_test, y_pred_lr_test, alpha=0.6, color='blue')
axes[0].plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2)
axes[0].set_xlabel('Preço Real (R$)')
axes[0].set_ylabel('Preço Predito (R$)')
axes[0].set_title(f'📈 Regressão Linear\nR² = {lr_r2_test:.3f}')
axes[0].grid(True, alpha=0.3)

# Random Forest
axes[1].scatter(y_test, y_pred_rf_test, alpha=0.6, color='green')
axes[1].plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2)
axes[1].set_xlabel('Preço Real (R$)')
axes[1].set_ylabel('Preço Predito (R$)')
axes[1].set_title(f'🌲 Random Forest\nR² = {rf_r2_test:.3f}')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Importância das features (Random Forest)
if len(X.columns) > 1:
    feature_importance = pd.DataFrame({
        'Feature': X.columns,
        'Importância': rf_model.feature_importances_
    }).sort_values('Importância', ascending=False)

    plt.figure(figsize=(10, 6))
    sns.barplot(data=feature_importance, x='Importância', y='Feature', palette='viridis')
    plt.title('🌲 Importância das Features - Random Forest')
    plt.xlabel('Importância')
    plt.tight_layout()
    plt.show()

    print("📊 Ranking de Importância das Features:")
    for idx, row in feature_importance.iterrows():
        print(f"   {row['Feature']}: {row['Importância']:.4f}")
else:
    print("📊 Apenas uma feature disponível para modelagem")

In [ ]:
# Análise de resíduos
residuals_lr = y_test - y_pred_lr_test
residuals_rf = y_test - y_pred_rf_test

fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Resíduos vs Predições - Linear
axes[0,0].scatter(y_pred_lr_test, residuals_lr, alpha=0.6, color='blue')
axes[0,0].axhline(y=0, color='r', linestyle='--')
axes[0,0].set_xlabel('Predições')
axes[0,0].set_ylabel('Resíduos')
axes[0,0].set_title('📈 Resíduos - Regressão Linear')
axes[0,0].grid(True, alpha=0.3)

# Histograma dos resíduos - Linear
axes[0,1].hist(residuals_lr, bins=20, alpha=0.7, color='blue', edgecolor='black')
axes[0,1].set_xlabel('Resíduos')
axes[0,1].set_ylabel('Frequência')
axes[0,1].set_title('📊 Distribuição dos Resíduos - Linear')

# Resíduos vs Predições - Random Forest
axes[1,0].scatter(y_pred_rf_test, residuals_rf, alpha=0.6, color='green')
axes[1,0].axhline(y=0, color='r', linestyle='--')
axes[1,0].set_xlabel('Predições')
axes[1,0].set_ylabel('Resíduos')
axes[1,0].set_title('🌲 Resíduos - Random Forest')
axes[1,0].grid(True, alpha=0.3)

# Histograma dos resíduos - Random Forest
axes[1,1].hist(residuals_rf, bins=20, alpha=0.7, color='green', edgecolor='black')
axes[1,1].set_xlabel('Resíduos')
axes[1,1].set_ylabel('Frequência')
axes[1,1].set_title('📊 Distribuição dos Resíduos - RF')

plt.tight_layout()
plt.show()

In [ ]:
# Função para fazer predições
def predict_price(model, scaler, **kwargs):
    """Função para predizer preço de um imóvel"""
    # Criar DataFrame com as features
    input_data = pd.DataFrame([kwargs])
    
    # Garantir que todas as features estão presentes
    for col in X.columns:
        if col not in input_data.columns:
            input_data[col] = 0  # Valor padrão
    
    # Reordenar colunas
    input_data = input_data[X.columns]
    
    # Fazer predição
    if model == rf_model:
        prediction = model.predict(input_data)[0]
    else:  # Linear Regression
        input_scaled = scaler.transform(input_data)
        prediction = model.predict(input_scaled)[0]
    
    return prediction

# Exemplo de predição
print("🔮 Exemplo de Predição:")
print("=" * 40)

# Usar valores médios das features como exemplo
example_features = {}
for col in X.columns:
    example_features[col] = X[col].mean()

print(f"📋 Features do exemplo:")
for feature, value in example_features.items():
    print(f"   {feature}: {value:.2f}")

# Predições
pred_lr = predict_price(lr_model, scaler, **example_features)
pred_rf = predict_price(rf_model, scaler, **example_features)

print(f"\n💰 Predições de Preço:")
print(f"   Regressão Linear: R$ {pred_lr:,.2f}")
print(f"   Random Forest: R$ {pred_rf:,.2f}")
print(f"   Preço médio real: R$ {y.mean():,.2f}")

In [ ]:
# Validação cruzada
print("🔄 Realizando Validação Cruzada...")

# Linear Regression
cv_scores_lr = cross_val_score(lr_model, X_train_scaled, y_train, cv=5, scoring='r2')
print(f"\n📈 Regressão Linear - CV:")
print(f"   R² médio: {cv_scores_lr.mean():.4f} (±{cv_scores_lr.std()*2:.4f})")
print(f"   R² por fold: {cv_scores_lr}")

# Random Forest
cv_scores_rf = cross_val_score(rf_model, X_train, y_train, cv=5, scoring='r2')
print(f"\n🌲 Random Forest - CV:")
print(f"   R² médio: {cv_scores_rf.mean():.4f} (±{cv_scores_rf.std()*2:.4f})")
print(f"   R² por fold: {cv_scores_rf}")

In [ ]:
# Resumo final
print("\n" + "="*60)
print("🎯 RESUMO FINAL DOS MODELOS")
print("="*60)

print(f"\n📊 Dataset:")
print(f"   Total de registros: {len(df_model)}")
print(f"   Features utilizadas: {len(X.columns)}")
print(f"   Treino/Teste: {len(X_train)}/{len(X_test)}")

print(f"\n🏆 Melhor Modelo: {best_model_name}")
if best_model_name == 'Random Forest':
    print(f"   R² Teste: {rf_r2_test:.4f}")
    print(f"   RMSE: R$ {rf_rmse_test:,.2f}")
    print(f"   MAE: R$ {rf_mae_test:,.2f}")
else:
    print(f"   R² Teste: {lr_r2_test:.4f}")
    print(f"   RMSE: R$ {lr_rmse_test:,.2f}")
    print(f"   MAE: R$ {lr_mae_test:,.2f}")

print(f"\n💡 Interpretação:")
if max(rf_r2_test, lr_r2_test) > 0.7:
    print("   ✅ Modelo com boa capacidade preditiva")
elif max(rf_r2_test, lr_r2_test) > 0.4:
    print("   ⚠️ Modelo com capacidade preditiva moderada")
else:
    print("   ❌ Modelo com baixa capacidade preditiva")

print("\n✅ Modelagem concluída com sucesso!")